In [ ]:
import os

import matplotlib
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from scipy.stats import circmean, circstd

from simpleredcal.fit_diagnostics import abs_residuals
from simpleredcal.plot_utils import antpos_map, df_heatmap, plot_res, \
plot_res_grouped, plot_res_heatmap
from simpleredcal.red_utils import find_opt_df, flt_ant_coords, RESPATH

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib as mpl

plot_figs = False
if plot_figs:
    mpl.rcParams['figure.dpi'] = 300

mpl.rc('font',**{'family':'serif','serif':['cm']})
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

In [ ]:
jd_time = 2458098.43869
pol = 'ee'
dist = 'cauchy'
rel_dir_path = os.path.join(RESPATH, 'rel_dfs')
opt_dir_path = os.path.join(RESPATH, 'opt_dfs')

In [ ]:
opt_df = find_opt_df(jd_time, pol, dist, opt_dir_path)

In [ ]:
with open(os.path.join(rel_dir_path, 'rel_df.{}.{}.md.pkl'.format(jd_time, pol)), 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(opt_df)
df

In [ ]:
test_freq = 600
test_tint = 0

resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)

In [ ]:
# check results for a given frequency & time integration
test_freq = 600
test_tint = 0

no_ants = md['no_ants']
no_deg_params = 4
resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)
test_gains_comps, test_deg_params, test_vis_comps = np.split(resx, [2*no_ants, \
                                                    2*no_ants+no_deg_params,])

test_gamps = test_gains_comps[::2]
test_gphases = test_gains_comps[1::2]
mean_test_amps = np.mean(test_gamps)
mean_test_phases = np.mean(test_gphases)

print('Gain amplitude average is: {}'.format(mean_test_amps))
print('Gain phase average is: {}'.format(mean_test_phases))

In [ ]:
antpos = flt_ant_coords(jd_time, md['antpos'])

In [ ]:
antpos_map(test_gamps, antpos, 'Gain amplitudes', center=1)
print('Gain amplitude solutions for frequency channel {} and time integration {} are: '\
      '\n{}'.format(test_freq, test_tint, test_gamps))

In [ ]:
antpos_map(test_gphases, antpos, 'Gain phases', center=0)
print('Gain phase solutions are:\n{}'.format(test_gphases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
plot_res(df, 'nit', clip=True)

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

In [ ]:
plot_res_heatmap(df, 'nit', clip=True, clip_pctile=96)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', clip=True, clip_pctile=97)

In [ ]:
plot_res_grouped(df, 'fun', logy=True)

In [ ]:
plot_res_heatmap(df, 'fun', clip=True, clip_pctile=98)

### Residuals

Look at both the real and imaginary of the the median absolute normalized residual

In [ ]:
df[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = df.apply(lambda row: \
pd.Series(abs_residuals(row['norm_residual'])), axis=1)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_Re', clip=True, clip_pctile=97)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_Im', clip=True, clip_pctile=97)

Combined residual by adding Re and Im components in quadrature

In [ ]:
df['med_abs_norm_res_comb'] = np.sqrt(df['med_abs_norm_res_Re']**2 + \
                                      df['med_abs_norm_res_Im']**2)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_comb', clip=True, clip_pctile=97)

### Stability of gains, visibility solutions and degenerate parameters

#### Gains

In [ ]:
gains_df = df.iloc[:, 5:2*no_ants+5]

gains_df['gamp_mean'] = gains_df.apply(lambda row: np.mean(row[:2*no_ants:2]), axis=1)
gains_df['gphase_mean'] = gains_df.apply(lambda row: np.mean(row[1:2*no_ants:2]), axis=1)

In [ ]:
plot_res(gains_df, 'gamp_mean', ylim=(0.9, 1.1), \
         title='Average gain amplitude for each frequency and time integration slice')

In [ ]:
# average gain phase for each frequency and time slice
plot_res(gains_df, 'gphase_mean', ylim=(-np.pi, np.pi), \
        title='Average gain amplitude for each frequency and time integration  slice')

##### Gain stability at test_freq {{test_freq}}

In [ ]:
gamps_df = gains_df.iloc[:, :2*no_ants:2]
gphases_df = gains_df.iloc[:, 1:2*no_ants:2]
gamps_df.columns = np.arange(no_ants) + 1
gphases_df.columns = gamps_df.columns

In [ ]:
df_heatmap(gamps_df.loc[test_freq, :], ybase=5,\
           title='Gain amplitudes for frequency channel {}'.format(test_freq),
           xlabel='Antenna',
           ylabel='Time integration')

In [ ]:
df_heatmap(gphases_df.loc[test_freq, :], ybase=5, center=0, cmap='bwr', vmin=-np.pi, vmax=np.pi, \
           title='Gain phases for frequency channel {}'.format(test_freq),
           xlabel='Antenna',
           ylabel='Time integration')

##### Gain stability across average over frequency channels 600-700

In [ ]:
std_rng = 3
chan_start = 600
chan_end = 700

# mean over the time integrations for each frequency
avg_gamps_tint = gamps_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].groupby(level=0).mean()
piv = pd.pivot_table(avg_gamps_tint, columns='freq')
vrng = np.ceil(np.std(piv.values)*std_rng*10)/10

df_heatmap(piv, xbase=10, ybase=5, cmap='bwr', center=1, vmin=1-vrng, vmax=1+vrng, \
           title='Mean (over time integrations) gain amplitudes', \
           xlabel='Frequency', \
           ylabel='Time integration')

In [ ]:
# # circular mean over the time integrations for each frequency
# circ_res = avg_gphases_tint.apply(lambda x: circmean(x, axis=0))
# avg_gphases_tint = pd.DataFrame(circ_res.values.tolist(), index=circ_res.index)

avg_gphases_tint = gphases_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].\
                   groupby(level=0).mean()
piv = pd.pivot_table(avg_gphases_tint, columns='freq')

df_heatmap(piv, xbase=10, ybase=5, cmap='bwr', center=0, vmin=-np.pi, vmax=np.pi, \
           title='Mean (over time integrations) gain phases', \
           xlabel='Frequency', \
           ylabel='Time integration')

##### Average gains across frequency channels 600-700 and all time integrations

In [ ]:
avg_gain_comps = gains_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].mean(axis=0).values[:2*no_ants]
avg_gamps = avg_gain_comps[::2]
avg_gphases = avg_gain_comps[1::2]

In [ ]:
antpos_map(avg_gamps, antpos, 'Average gain amplitudes', center=1)

In [ ]:
antpos_map(avg_gphases, antpos, 'Average gain phases', center=0)

##### Standard deviation of gains across frequency channels 600-700 and all time integrations

In [ ]:
avg_gain_comps = gains_df.loc[pd.IndexSlice[chan_start:chan_end, :], :].std(axis=0).values[:2*no_ants]
std_gamps = avg_gain_comps[::2]
std_gphases = avg_gain_comps[1::2]
circ_std_gphases = gains_df.iloc[:, 1:2*no_ants:2].apply(lambda col: circstd(col), \
                                                         axis=0).values

In [ ]:
antpos_map(avg_gamps, antpos, 'Standard deviation of gain amplitudes', cmap='Oranges')

In [ ]:
antpos_map(circ_std_gphases, antpos, 'Circular standard deviation of gain phases', \
           cmap='Oranges')

#### Visibility solutions

In [ ]:
no_resid_p = 5 # number of residual parameters in df dataframe
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = df.iloc[:, 2*no_ants+no_min_p+no_deg_params:-no_resid_p]

no_unq_bls = md['no_unq_bls']
vis_df['vamp_mean'] = vis_df.apply(lambda row: np.mean(row[:2*no_unq_bls:2]), axis=1)
vis_df['vphase_mean'] = vis_df.apply(lambda row: np.mean(row[1:2*no_unq_bls:2]), \
                                     axis=1)

In [ ]:
plot_res(vis_df, 'vamp_mean', ylim=(-0.01, 0.01))

In [ ]:
plot_res_heatmap(vis_df, 'vamp_mean', clip=True, clip_pctile=97)

In [ ]:
plot_res_heatmap(vis_df, 'vamp_mean', vmax=0.01, vmin=-0.01)

In [ ]:
plot_res(vis_df, 'vphase_mean', ylim=(-np.pi, np.pi))

##### Visibilities across baselines at test_integration {{test_tint}}

In [ ]:
visC_df = vis_df.iloc[:, :-2]
visC_tint_df = visC_df.loc[pd.IndexSlice[:, test_tint], :].droplevel(level=1)
visamp_tint_df = visC_tint_df.iloc[:, ::2]
visphase_tint_df = visC_tint_df.iloc[:, 1::2]
visamp_tint_df.columns = np.arange(no_unq_bls)
visphase_tint_df.columns = visamp_tint_df.columns

In [ ]:
piv = piv = pd.pivot_table(visamp_tint_df, columns='freq')
vmax = np.nanpercentile(piv.values, 99)
vmin = np.nanpercentile(piv.values, 1)
df_heatmap(piv, xbase=50, ybase=5, vmin=vmin, vmax=vmax, \
           title='Visibility amplitudes for time_integration {}'.format(test_tint), \
           xlabel='Frequency', \
           ylabel='Redundant Baseline Group')

In [ ]:
piv = pd.pivot_table(visphase_tint_df, columns='freq')
vmax = np.nanpercentile(piv.values, 99)
vmin = np.nanpercentile(piv.values, 1)
df_heatmap(piv, xbase=50, ybase=5, center=0, cmap='bwr', vmax=vmax, vmin=vmin, \
           title='Visibility phases for time_integration {}'.format(test_tint), \
           xlabel='Frequency', \
           ylabel='Redundant Baseline Group')

#### Degenerate parameters

In [ ]:
degp_df = df.iloc[:, 2*no_ants+no_min_p:2*no_ants+no_min_p+no_deg_params]
degp_df.columns = ['overall_amp', 'overall_phase', 'phase_grad_x', 'phase_grad_y']

In [ ]:
plot_res(degp_df, 'overall_amp')

In [ ]:
vmin = np.nanpercentile(degp_df['overall_amp'].values, 1)
plot_res_heatmap(degp_df, 'overall_amp', vmin=vmin)

In [ ]:
plot_res(degp_df, 'phase_grad_x', clip=True, clip_pctile=97)

In [ ]:
plot_res_heatmap(degp_df, 'phase_grad_x', clip=True, clip_pctile=96)

In [ ]:
plot_res(degp_df, 'phase_grad_y', clip=True, clip_pctile=97)

In [ ]:
plot_res_heatmap(degp_df, 'phase_grad_y', cmap='bwr', center=0, clip=True, clip_pctile=96)